In [7]:
import pandas as pd
data = pd.read_csv("/Users/ethanschultz/Downloads/AmesHousing.csv")
data.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,...,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,...,Y,SBrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,...,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,...,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,NaN,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,...,Y,SBrkr,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,...,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [14]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures,OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score
import numpy as np

In [9]:
X = data.drop("SalePrice", axis = 1)
y = data["SalePrice"]



X_train, X_test, y_train, y_test = train_test_split(X, y)

In [15]:
# Model 1
ct = ColumnTransformer(
  [
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])
  ],
  remainder = "drop"
)


lr_pipeline = Pipeline(
  [("preprocessing", ct),
  ("linear_regression", LinearRegression())]
)

lr_pipeline

lr_fitted = lr_pipeline.fit(X_train, y_train)
train_pred = lr_fitted.predict(X_train)
test_pred = lr_fitted.predict(X_test)
test_mse = mean_squared_error(y_test, test_pred)
rsme = np.sqrt(test_mse)
test_r2 = r2_score(y_test, test_pred)
intercept = lr_fitted.named_steps['linear_regression'].intercept_
coefficient = lr_fitted.named_steps['linear_regression'].coef_

print("Test MSE:", test_mse)
print("Test R^2:", test_r2)
print("RSME:", rsme)
print("intercept:", intercept)
print("coefficients:", coefficient)
# Cross val for second part
scores = cross_val_score(lr_pipeline, X, y, scoring='neg_mean_squared_error', cv=5)
mse = -scores
mse_average = np.mean(mse)
rmse_average = np.sqrt(mse_average)
rmse_average



Test MSE: 3408517128.5818048
Test R^2: 0.4276458947226658
RSME: 58382.507042621975
intercept: 180900.64087391898
coefficients: [ 72620.85986411 -17537.07343714]
Test MSE: 3408517128.5818048
Test R^2: 0.4276458947226658
RSME: 58382.507042621975
intercept: 180900.64087391898
coefficients: [ 72620.85986411 -17537.07343714]


56001.24023779208

In [16]:
# Model 2
ct1 = ColumnTransformer(
  [
    ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"]),    
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"])

  ],
  remainder = "drop"
)


lr_pipeline = Pipeline(
  [("preprocessing", ct1),
  ("linear_regression", LinearRegression())]
)

lr_pipeline

lr_fitted = lr_pipeline.fit(X_train, y_train)
train_pred = lr_fitted.predict(X_train)
test_pred = lr_fitted.predict(X_test)
test_mse = mean_squared_error(y_test, test_pred)
rsme = np.sqrt(test_mse)
test_r2 = r2_score(y_test, test_pred)
intercept = lr_fitted.named_steps['linear_regression'].intercept_
coefficient = lr_fitted.named_steps['linear_regression'].coef_

print("Test MSE:", test_mse)
print("Test R^2:", test_r2)
print("RSME:", rsme)
print("intercept:", intercept)
print("coefficients:", coefficient)
# Cross val for second part
scores = cross_val_score(lr_pipeline, X, y, scoring='neg_mean_squared_error', cv=5)
mse = -scores
mse_average = np.mean(mse)
rmse_average = np.sqrt(mse_average)
rmse_average


Test MSE: 3188270306.2169166
Test R^2: 0.4646294767905902
RSME: 56464.770487596215
intercept: -6916377174512560.0
coefficients: [ 6.75445443e+04 -1.02599241e+04  6.91637717e+15  6.91637717e+15
  6.91637717e+15  6.91637717e+15  6.91637717e+15]
Test MSE: 3188270306.2169166
Test R^2: 0.4646294767905902
RSME: 56464.770487596215
intercept: -6916377174512560.0
coefficients: [ 6.75445443e+04 -1.02599241e+04  6.91637717e+15  6.91637717e+15
  6.91637717e+15  6.91637717e+15  6.91637717e+15]


54305.840806452674

In [17]:
# Model 3
ct_pre = ColumnTransformer(
    [
        ("standardize", StandardScaler(), ["Gr Liv Area"]),
        ("dummify", OneHotEncoder(sparse_output=False), ["Bldg Type"])
    ],
    remainder="drop"
).set_output(transform="pandas")

ct_inter = ColumnTransformer(
  [
    ("interaction", PolynomialFeatures(interaction_only = True), ["standarize__Gr Liv Area", "dummify_Bldg Type_1Fam"]),
    ("interaction", PolynomialFeatures(interaction_only = True), ["standarize__Gr Liv Area", "dummify_Bldg Type_2fmCon"]),
    ("interaction", PolynomialFeatures(interaction_only = True), ["standarize__Gr Liv Area", "dummify_Bldg Type_Duplex"]),
    ("interaction", PolynomialFeatures(interaction_only = True), ["standarize__Gr Liv Area", "dummify_Bldg Type_Twnhs"]),
    ("interaction", PolynomialFeatures(interaction_only = True), ["standarize__Gr Liv Area", "dummify_Bldg Type_TwnhsE"]),
    ],
  remainder = "drop"
).set_output(transform = "pandas")

# Step 3: Build the pipeline
lr_pipeline = Pipeline(
    [
        ("preprocessing", ct_pre),  
        ("interaction", PolynomialFeatures(interaction_only=True, include_bias=False)),
        ("linear_regression", LinearRegression())  
    ]
)

lr_fitted = lr_pipeline.fit(X_train, y_train)
train_pred = lr_fitted.predict(X_train)
test_pred = lr_fitted.predict(X_test)
test_mse = mean_squared_error(y_test, test_pred)
rsme = np.sqrt(test_mse)
test_r2 = r2_score(y_test, test_pred)
intercept = lr_fitted.named_steps['linear_regression'].intercept_
coefficient = lr_fitted.named_steps['linear_regression'].coef_

print("Test MSE:", test_mse)
print("Test R^2:", test_r2)
print("RSME:", rsme)
print("intercept:", intercept)
print("coefficients:", coefficient)
# Cross val for second part
scores = cross_val_score(lr_pipeline, X, y, scoring='neg_mean_squared_error', cv=5)
mse = -scores
mse_average = np.mean(mse)
rmse_average = np.sqrt(mse_average)
rmse_average


Test MSE: 3122601537.0341063
Test R^2: 0.47565649769511176
RSME: 55880.24281473825
intercept: -9.107130234428082e+17
coefficients: [-1.29701659e+17  9.10713023e+17  9.10713023e+17  9.10713023e+17
  9.10713023e+17  9.10713023e+17  1.29701659e+17  1.29701659e+17
  1.29701659e+17  1.29701659e+17  1.29701659e+17  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00]
Test MSE: 3122601537.0341063
Test R^2: 0.47565649769511176
RSME: 55880.24281473825
intercept: -9.107130234428082e+17
coefficients: [-1.29701659e+17  9.10713023e+17  9.10713023e+17  9.10713023e+17
  9.10713023e+17  9.10713023e+17  1.29701659e+17  1.29701659e+17
  1.29701659e+17  1.29701659e+17  1.29701659e+17  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00]


53562.605169027745

In [18]:
# Model 4
ct_pre = ColumnTransformer(
    [
        ('encode', OneHotEncoder(sparse_output=False), ['Bldg Type']),
        ('poly_size', Pipeline([
           ("standardize", StandardScaler()),
           ("poly", PolynomialFeatures(degree = 5, include_bias=False))
        ]), ['Gr Liv Area']),
        ('poly_rooms', Pipeline([
            ("standardize", StandardScaler()),
            ("poly", PolynomialFeatures(degree = 5, include_bias=False))    
        ]), ["TotRms AbvGrd"])
    ],
    remainder="drop"
).set_output(transform="pandas")


ct_pre.fit_transform(X_train)

# ct_poly.fit_transform(ct_pre.fit_transform(X_train))

lr_pipeline = Pipeline([
    ('preprocessing', ct_pre),
    ('linear_regression', LinearRegression())
])

lr_fitted = lr_pipeline.fit(X_train, y_train)
train_pred = lr_fitted.predict(X_train)
test_pred = lr_fitted.predict(X_test)
test_mse = mean_squared_error(y_test, test_pred)
rsme = np.sqrt(test_mse)
test_r2 = r2_score(y_test, test_pred)
intercept = lr_fitted.named_steps['linear_regression'].intercept_
coefficient = lr_fitted.named_steps['linear_regression'].coef_

print("Test MSE:", test_mse)
print("Test R^2:", test_r2)
print("RSME:", rsme)
print("intercept:", intercept)
print("coefficients:", coefficient)
# Cross val for second part
scores = cross_val_score(lr_pipeline, X, y, scoring='neg_mean_squared_error', cv=5)
mse = -scores
mse_average = np.mean(mse)
rmse_average = np.sqrt(mse_average)
rmse_average


Test MSE: 3039134691.4888096
Test R^2: 0.48967215022089994
RSME: 55128.34744021273
intercept: 162186.9735904321
coefficients: [ 1.94879662e+04 -2.93733205e+04 -3.19032940e+04 -8.69903401e+02
  4.26585516e+04  7.08984744e+04 -1.15164449e+03 -1.24893659e+03
  9.36479112e+02 -1.07005569e+02 -1.45977442e+04  1.74773268e+03
  1.99128460e+03 -5.70719776e+02 -4.09150217e+01]
Test MSE: 3039134691.4888096
Test R^2: 0.48967215022089994
RSME: 55128.34744021273
intercept: 162186.9735904321
coefficients: [ 1.94879662e+04 -2.93733205e+04 -3.19032940e+04 -8.69903401e+02
  4.26585516e+04  7.08984744e+04 -1.15164449e+03 -1.24893659e+03
  9.36479112e+02 -1.07005569e+02 -1.45977442e+04  1.74773268e+03
  1.99128460e+03 -5.70719776e+02 -4.09150217e+01]


55470.14448606193

From each of the four models the one with the lowest RMSE was mdeol three with size, building type, and their interaction. After doing cross validation for each of these four models as well our results were confirmed and model three still had the lowest RSME and it was even lower than when we didn't use cross validation.

Part 3

In [26]:
from sklearn.model_selection import GridSearchCV

ct_poly = ColumnTransformer(
  [
    ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"]),
    ("polynomial_area", PolynomialFeatures(), ["Gr Liv Area"]),
    ("polynomial_rooms", PolynomialFeatures(), ["TotRms AbvGrd"])
  ],
  remainder = "drop"
)

lr_pipeline_poly = Pipeline(
  [("preprocessing", ct_poly),
  ("linear_regression", LinearRegression())]
).set_output(transform="pandas")

degrees = {'preprocessing__polynomial_area__degree': np.arange(1, 10), 'preprocessing__polynomial_rooms__degree': np.arange(1, 10)}

gscv = GridSearchCV(lr_pipeline_poly, degrees, cv = 5, scoring='r2')

X = data[["Gr Liv Area", "TotRms AbvGrd","Bldg Type"]]
y = data["SalePrice"]

gscv_fitted = gscv.fit(X, y)

gscv_fitted.cv_results_['mean_test_score']
results = gscv.cv_results_
df_results = pd.DataFrame({
    "degree_area": results['param_preprocessing__polynomial_area__degree'],
    "degree_rooms": results['param_preprocessing__polynomial_rooms__degree'],
    "mean_test_score": results['mean_test_score']
})
df_results_sorted = df_results.sort_values(by="mean_test_score", ascending=False)
df_results_sorted

,degree_area,degree_rooms,mean_test_score
18,3,1,0.557641
30,4,4,0.556860
19,3,2,0.556857
31,4,5,0.556469
33,4,7,0.554093
...,...,...,...
75,9,4,-4.545595
76,9,5,-4.545595
77,9,6,-4.545595
78,9,7,-4.545595


#1
The model that preformed the best was when house size was in a polynomial of 3 and rooms was in a polynomial of 1. After the corss validation the r^2 was 0.558.
#2
One downside that I noticed is that by far this code chunk took the most time to run out of anything we have tried in the class so far. Even though it was only a few seconds still this was a relatively simple model and at times there will be much more complex models and that time will only increase. Choosing lower polynomial features or the range would produce far less models and think more intuitevly about what changes actually make sense with given variables so you don't have to try them all.